In [1]:
import pandas as pd
import numpy as np


def replace_item_id_with_its_one_hot_value(item_id, table):
    if len(table[table['item_id'] == item_id].tags) == 0:
        return np.nan
    return table[table['item_id'] == item_id].tags.values[0]  # this step here takes a lot of time


In [19]:
item_one_hot_table = pd.read_json('numeric_values_tags.json')
aus_user = pd.read_csv("aus_user_item.csv",index_col=[0])
aus_user_with_onehot = pd.merge(aus_user, item_one_hot_table, on="item_id", how="left")
aus_user_with_onehot = aus_user_with_onehot.dropna(how = "any")

,user_id,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,tags
0,76561197970982479,76561197970982479,10,Counter-Strike,6,0,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0,0,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,76561197970982479,76561197970982479,30,Day of Defeat,7,0,"[0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,76561197970982479,76561197970982479,40,Deathmatch Classic,0,0,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,76561197970982479,76561197970982479,50,Half-Life: Opposing Force,0,0,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
5153203,76561198329548331,76561198329548331,227940,Heroes & Generals,43,43,"[0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, ..."
5153204,76561198329548331,76561198329548331,346330,BrainBread 2,0,0,"[0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, ..."
5153205,76561198329548331,76561198329548331,373330,All Is Dust,0,0,"[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, ..."
5153206,76561198329548331,76561198329548331,388490,One Way To Die: Steam Edition,3,3,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [21]:
aus_user_with_onehot["weighted_one_hot"] = aus_user_with_onehot.apply(
    lambda x: x.playtime_forever * 0.04 * np.array(x.tags, dtype=float) + 
              x.playtime_2weeks * 0.1 * np.array(x.tags, dtype=float),
    axis=1)
print("finish weighted one hot")

finish weighted one hot


In [42]:
x = aus_user_with_onehot.groupby('steam_id')["weighted_one_hot"].apply(list)

In [43]:
all_weighted_data = x.apply(lambda x: np.mean(x, axis=0))


In [44]:
with open("aus_user_feature.json", "w") as f:
    all_weighted_data.to_json(f)

1 2
5 6


IndexError: pop index out of range